<a href="https://colab.research.google.com/github/RonanD10/Tennis-Match-Prediction/blob/main/betting_odds_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

import torch
import torch.nn as nn
import torch.nn.functional as F

from google.colab import drive, files
drive.mount('/content/drive')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

Mounted at /content/drive


In [222]:
bet_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/betting_data.csv')
features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_random_new.csv')
all_matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')

<ipython-input-222-c72a9455db40>:1: DtypeWarning: Columns (15,21,22,28,31,47,48,52) have mixed types. Specify dtype option on import or set low_memory=False.
  bet_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/betting_data.csv')
<ipython-input-222-c72a9455db40>:3: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [6]:
bet_df.shape

(54908, 55)

In [12]:
bet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54908 entries, 0 to 54907
Data columns (total 55 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  54908 non-null  int64  
 1   ATP         54908 non-null  int64  
 2   AvgL        25354 non-null  float64
 3   AvgW        25354 non-null  float64
 4   B&WL        1097 non-null   float64
 5   B&WW        1097 non-null   float64
 6   B365L       46263 non-null  float64
 7   B365W       46240 non-null  float64
 8   Best of     54908 non-null  int64  
 9   CBL         17506 non-null  float64
 10  CBW         17506 non-null  float64
 11  Comment     54908 non-null  object 
 12  Court       54908 non-null  object 
 13  Date        54908 non-null  object 
 14  EXL         39416 non-null  float64
 15  EXW         39411 non-null  object 
 16  GBL         5055 non-null   float64
 17  GBW         5055 non-null   float64
 18  IWL         13358 non-null  float64
 19  IWW         13358 non-nul

In [223]:
bet_data = bet_df[['Loser', 'Winner', 'B365W', 'B365L', 'WRank', 'LRank']]
bet_data = bet_data.dropna(axis=0)
bet_data['Winner'] = bet_data['Winner'].map(lambda x: x.split()[0])
bet_data['Loser'] = bet_data['Loser'].map(lambda x: x.split()[0])
bet_data['WRank'] = bet_data['WRank'].astype(float)
bet_data['LRank'] = bet_data['LRank'].astype(float)
bet_data.reset_index(drop=True, inplace=True)

In [139]:
bet_data.head()

,Loser,Winner,B365W,B365L,WRank,LRank
0,Pretzsch A.,Johansson T.,1.100,6.500,7,133
1,Bjorkman J.,Kafelnikov Y.,1.444,2.625,6,56
2,Moya C.,Koubek S.,2.625,1.444,41,20
3,Stepanek R.,Pavel A.,1.533,2.375,21,150
4,Stoliarov A.,Sampras P.,1.143,5.000,13,107


In [84]:
all_matches.columns

Index(['index', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'tourney_date', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'score', 'best_of', 'round', 'minutes', 'w_ace', 'w_df',
       'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved',
       'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon',
       'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'winner_rank',
       'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'winner_hard_elo_before', 'loser_hard_elo_before',
       'winner_hard_elo_after', 'loser_hard_elo_after',
       'winner_clay_elo_before', 'loser_clay_elo_before',
       'winner_clay_elo_after', 'loser_clay_elo_after',
       'winner_grass_elo_before', 'loser_grass_elo_before',
 

In [130]:
features.columns

Index(['A_rank', 'B_rank', 'A_elo_surface', 'B_elo_surface', 'A_elo', 'B_elo',
       'A_avged_elo', 'B_avged_elo', 'A_12month_form', 'B_12month_form',
       'A_12month_form_surface', 'B_12month_form_surface', 'slam_round',
       'A_won', 'elo_difference'],
      dtype='object')

In [225]:
# Add features player names from ranking (in future, preserve this during feature engineering!)

features['A_name'] = ''
features['B_name'] = ''

j = 0
for i in range(len(features)):
    a_rank = features.loc[i, 'A_rank']
    b_rank = features.loc[i, 'B_rank']

    while j < len(all_matches):
        if (all_matches.loc[j, 'winner_rank'] == a_rank) and (all_matches.loc[j,'loser_rank'] == b_rank):
            features.loc[i, 'A_name'] = all_matches.loc[j, 'winner_name']
            features.loc[i, 'B_name'] = all_matches.loc[j, 'loser_name']
            j += 1
            break
        elif (all_matches.loc[j, 'winner_rank'] == b_rank) and (all_matches.loc[j, 'loser_rank'] == a_rank):
            features.loc[i, 'B_name'] = all_matches.loc[j, 'winner_name']
            features.loc[i, 'A_name'] = all_matches.loc[j, 'loser_name']
            j += 1
            break
        else:
            j += 1
            continue

In [228]:
bet_data

,Loser,Winner,B365W,B365L,WRank,LRank
0,Pretzsch,Johansson,1.100,6.500,7.0,133.0
1,Bjorkman,Kafelnikov,1.444,2.625,6.0,56.0
2,Moya,Koubek,2.625,1.444,41.0,20.0
3,Stepanek,Pavel,1.533,2.375,21.0,150.0
4,Stoliarov,Sampras,1.143,5.000,13.0,107.0
...,...,...,...,...,...,...
46137,Tsitsipas,Nadal,1.440,2.750,1.0,6.0
46138,Medvedev,Zverev,1.900,1.900,7.0,4.0
46139,Federer,Tsitsipas,3.500,1.300,6.0,3.0
46140,Zverev,Thiem,1.800,2.000,5.0,7.0


In [227]:
features['A_name'] = features['A_name'].map(lambda x: x.split()[-1])
features['B_name'] = features['B_name'].map(lambda x: x.split()[-1])

In [232]:
len(features)

144639

In [241]:
# Add betting odds to features

features['A_odds'] = 0
features['B_odds'] = 0

for i in range(len(bet_data)):

    if i % 1000 == 0:
        clear_output(wait=True)
        print(i/len(bet_data))

    w_name = bet_data.loc[i, 'Winner']
    l_name = bet_data.loc[i, 'Loser']
    w_rank = bet_data.loc[i, 'WRank']
    l_rank = bet_data.loc[i, 'LRank']
    w_odds = bet_data.loc[i, 'B365W']
    l_odds = bet_data.loc[i, 'B365L']

    match_row_a = features[(features['A_name'] == w_name) & (features['B_name'] == l_name) & (features['A_rank'] == w_rank) & (features['B_rank'] == l_rank)]
    match_row_b = features[(features['A_name'] == l_name) & (features['B_name'] == w_name) & (features['A_rank'] == l_rank) & (features['B_rank'] == w_rank)]

    if len(match_row_a) == 1:
        features.loc[match_row_a.index, 'A_odds'] = w_odds
        features.loc[match_row_a.index, 'B_odds'] = l_odds

    elif len(match_row_b) == 1:
        features.loc[match_row_b.index, 'A_odds'] = l_odds
        features.loc[match_row_b.index, 'B_odds'] = w_odds

    else:
        continue


0.9969225434528196


In [247]:
features[features['A_odds'] > 0].describe()

,A_rank,B_rank,A_elo_surface,B_elo_surface,A_elo,B_elo,A_avged_elo,B_avged_elo,A_12month_form,B_12month_form,A_12month_form_surface,B_12month_form_surface,slam_round,A_won,elo_difference,A_odds,B_odds
count,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000
mean,72.949605,72.010718,1663.295161,1663.292722,1730.723913,1730.226902,1697.009537,1696.759812,0.514894,0.514026,0.512100,0.512122,0.367462,0.496989,166.517842,2.656266,2.669417
std,94.292103,90.849886,158.780401,157.937295,169.673257,167.885310,159.169704,157.869613,0.156883,0.155329,0.191819,0.191026,0.919028,0.499997,133.641413,2.615857,2.715300
min,1.000000,1.000000,1312.653000,1290.154000,1376.282000,1361.059000,1385.056500,1366.645000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,24.000000,24.000000,1539.386500,1540.426000,1606.745000,1606.999000,1577.491500,1578.204000,0.428571,0.428571,0.411765,0.416667,0.000000,0.000000,63.615000,1.390000,1.390000
50%,52.000000,52.000000,1637.346000,1638.063000,1711.050000,1712.119000,1674.091000,1674.890000,0.516667,0.517241,0.529412,0.529412,0.000000,0.000000,137.074000,1.830000,1.830000
75%,89.000000,89.000000,1751.020000,1749.665000,1827.950000,1826.674000,1785.596750,1784.249750,0.612903,0.611111,0.638593,0.636364,0.000000,1.000000,235.795500,2.870000,2.750000
max,1890.000000,2159.000000,2382.061000,2383.940000,2442.724000,2463.437000,2402.244000,2423.688500,1.000000,0.967033,1.000000,1.000000,7.000000,1.000000,962.572000,51.000000,67.000000


In [248]:
features.to_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_odds_random.csv')

In [249]:
features_without_odds = features.drop(['A_odds', 'B_odds'], axis=1)
features_without_odds.to_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_names_random.csv')

In [250]:
features_with_odds = features[features['A_odds'] > 0]
features_with_odds.to_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_with_odds_random.csv')

In [252]:
features_with_odds.reset_index(drop=True, inplace=True)

In [253]:
features_with_odds

,A_rank,B_rank,A_elo_surface,B_elo_surface,A_elo,B_elo,A_avged_elo,B_avged_elo,A_12month_form,B_12month_form,A_12month_form_surface,B_12month_form_surface,slam_round,A_won,elo_difference,A_name,B_name,A_odds,B_odds
0,176.0,98.0,1464.629,1483.856,1516.231,1494.582,1490.4300,1489.2190,0.222222,0.375000,0.000000,0.500000,0,0,21.649,Ilie,Morrison,2.250,1.571
1,103.0,73.0,1507.343,1392.427,1578.613,1537.001,1542.9780,1464.7140,0.321429,0.400000,0.333333,0.666667,0,0,41.612,Llodra,Spadea,1.727,2.000
2,117.0,142.0,1493.451,1601.524,1595.061,1747.573,1544.2560,1674.5485,0.366667,0.500000,0.500000,0.500000,0,1,152.512,Lee,Larsson,2.625,1.444
3,111.0,281.0,1601.789,1500.000,1562.030,1517.104,1581.9095,1508.5520,0.500000,0.250000,0.750000,0.000000,0,1,44.926,Godwin,Gasquet,1.533,2.375
4,6.0,56.0,1661.571,1673.790,1752.723,1700.134,1707.1470,1686.9620,0.652174,0.470588,0.833333,0.555556,0,1,52.589,Kafelnikov,Bjorkman,1.444,2.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40674,8.0,5.0,1725.112,1964.696,1900.244,2049.208,1812.6780,2006.9520,0.651515,0.708333,0.527778,0.682927,0,1,148.964,Berrettini,Thiem,3.200,1.360
40675,3.0,6.0,2124.194,1921.724,2160.654,1985.868,2142.4240,1953.7960,0.814286,0.687500,0.804348,0.685185,0,0,174.786,Federer,Tsitsipas,1.300,3.500
40676,5.0,7.0,1939.133,1906.372,2026.740,1929.922,1982.9365,1918.1470,0.708333,0.643836,0.682927,0.659091,0,1,96.818,Thiem,Zverev,1.800,2.000
40677,5.0,6.0,1953.629,1946.119,2038.393,2009.301,1996.0110,1977.7100,0.708333,0.687500,0.682927,0.685185,0,0,29.092,Thiem,Tsitsipas,1.800,2.000
